# STG Pedidos

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


In [2]:
# Importando as bibliotecas 

import pandas as pd 
import sys
import os
from sqlalchemy import text
from sqlalchemy import create_engine
from dotenv import load_dotenv
from utils.acessobanco import acessobanco
# from helpers import truncate_table



In [3]:
# Carrega variáveis de ambiente de arquivo específico para conexão com o banco
load_dotenv(dotenv_path="/home/danielpedro/Python/Projetos/SuperStorePY/Bancos/acessobanco.env")

True

In [4]:
# Cria engine de conexão com o banco PostgreSQL
engine = acessobanco()

In [5]:
df_vendedores = pd.read_excel('/home/danielpedro/Python/Projetos/SuperStorePY/Dataset/Sample - Superstore.xlsx', sheet_name='People')


In [6]:
df_vendedores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Person  4 non-null      object
 1   Region  4 non-null      object
dtypes: object(2)
memory usage: 196.0+ bytes


In [7]:
df_vendedores.head()

,Person,Region
0,Anna Andreadi,West
1,Chuck Magee,East
2,Kelly Williams,Central
3,Cassandra Brandow,South


In [20]:
df_vendedores = df_vendedores.rename(columns={ 
    "Person": "nome_vendedor", 
    "Region":"regiao"})

In [21]:
df_vendedores.head()

,nome_vendedor,regiao
0,Anna Andreadi,West
1,Chuck Magee,East
2,Kelly Williams,Central
3,Cassandra Brandow,South


In [22]:
tabela = 'vendedores'
schema = 'stg'

try:
    with engine.begin() as conn:
        print(f'Limpando tabela {schema}.{tabela}...')
        conn.execute(text(f'TRUNCATE TABLE {schema}.{tabela}'))
        print(f'Tabela {schema}.{tabela} truncada com sucesso!')
        
except Exception as e:
    print(f'Erro ao truncar a tabela {schema}.{tabela}: {e}')
    print('Detalhes do erro:', str(e))

Limpando tabela stg.vendedores...
Tabela stg.vendedores truncada com sucesso!


In [23]:
# Gravando um os dados na tabela 

df_vendedores.to_sql(
    name='vendedores',
    con=engine,
    schema='stg',
    if_exists='append',
    index=False,         # não envia o índice do DataFrame
    method="multi",      # insere em lote (batch)
    chunksize=5000       # 5.000 linhas por inserção
)

4